[Accueil](../../../index.ipynb) > [Sommaire de Terminale](../../index.ipynb)

# Sécurisation des communications


La science de la sécurisation des communications s'appelle la **cryptologie** (Science du secret) qui est composée de :
- la **cryptographie** ("Ecriture secrète)"
- la **crytanalyse** ("Analyse des mécanismes de cryptographie")

La sécurisation doit permettre :
- La **confidentialité** du message échangé.
- l'**authenticité** ("qui est l'expéditeur ?")
- l'**intégrité** ("y a t-il eu modification?")

## Vocabulaire

- **Coder** : Représenter l'information pour un ensemble de signes.
- **Décoder** : Interpréter un ensemble de signes pour comprendre l'information.
- **Chiffrer** : Rendre incompréhensible un code à l'aide d'une clé de chiffrement.
- **Déchiffrer** : Retrouver le code originel d'un message chiffré à l'aide d'une clé de déchiffrement.
- **Décrypter** : Retrouver les signes originaux à partir d’un message chiffré, sans utiliser de clé de déchiffrement.


##  Chiffrement symétrique
On parle de **chiffrement symétrique** (ou chiffrement à **clé symétrique**) lorsqu'une même clé sert à chiffrer et déchiffrer un message.

[Article Wikipedia sur le chiffrement symétrique](https://fr.wikipedia.org/wiki/Cryptographie_sym%C3%A9trique)

Il existe de nombreux chiffrements qui ont évolué en complexité au cours du temps.

### Code de César

Utilisé par Jules César dans ces correspondances secrêtes, cette méthode de chiffrement est très simple. Elle consiste à décaler chaque lettre du texte d'origine par une autre lettre à distance fixe. Si le décalage dépasse la lettre Z on reprend au début. Il s'agit donc d'une permutation circulaire des lettres de l'alphabet.

Voir l'article [Wikipedia](https://fr.wikipedia.org/wiki/Chiffrement_par_d%C3%A9calage).
<figure>
<img src="img/caesar3.svg" alt="Chiffrement de César" title="Chiffrement de César">
<figcaption>Exemple d'un décalage de 3 dans le chiffrement de César.</figcaption>
</figure>

Evidemment ce genre de code nous apparait aujourd'hui trivial à décoder. En utilisant uniquement les 26 lettres de l'alphabet, il existe uniquement 25 clés possibles.

**EXERCICE**

- Ecrire une *fonction encode_cesar(text, cle)* qui prend en paramètre un texte (string) et une clé (entier) et qui retourne une chaine de caractère codée par le chiffrement de César.
- Ecrire une *fonction decode_cesar(text, cle)* qui prend en paramètre un texte codé (string) et une clé (entier) et qui retourne une chaine de caractère décodée.

Vous pourrez utiliser la fonction ci- dessous qui supprime les accents afin de se limiter aux caractères ASCII.

In [27]:
import unicodedata, string

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    #no_punctuation = only_ascii.decode("utf-8").translate(str.maketrans('', '', string.punctuation))
    return only_ascii.decode("utf-8")

text = remove_accents("Ceci est un texte en français, il possède donc des accents comme le é, à, ô et des lettres qui ne figurent pas dans le code ASCII.")
print(text)

Ceci est un texte en francais, il possede donc des accents comme le e, a, o et des lettres qui ne figurent pas dans le code ASCII.


### Chiffrement par substitution

Dans cette méthode le décalage n'est pas constant. On substitue chaque lettre du texte par une autre lettre de l'alphabet.

**Exemple**:

ABCDEFGHIJKLMNOPQRSTUVWXYZ

AZERTYUIOPQSDFGHJKLMWXCVBN

En utilisant uniquement les lettres majuscules on obtient donc 26! possibilités soit environ $4 \times 10^{26}$ possibilités.

Cependant ce chiffrement ne résiste pas à la **méthode des fréquences des lettres** qui permet de "casser" les clés de cryptage.

### Chiffrement de Vigénère (XVI siècle)

Cette méthode a été mise au point pour contrer la méthode des fréquences des lettres. La clé donne le décalage à effectuer, celui devient donc variable.

**Exemple**

Le message est "SECRET" et la clé est "BCD" on applique donc un décalage de 1, 2 et 3.

"S**E**CR**E**T**E**" devient donc "T**G**FS**G**W**F**". On remarque que la lettre E n'est pas codée toujours par le même symbole.

### Chiffrement de Vernam (XX siècle)

Il s'agit d'un système parfaitement sûr. [Claude Shannon](https://fr.wikipedia.org/wiki/Claude_Shannon) a démontré que ce système est inviolable si on respecte les 3 règles de Vernam:

 - La clé doit être aussi longue le message;
 - Les caractères de la clé doivent être choisie de façon aléatoire;
 - La clé ne doit être utilisée qu'une seule fois (on parle de masque jetable).
 
Une attaque par force brute est ici imposible : on aurait tous les messages possibles.
 
 **Exemple**
 
 - Le message est "HELLO"
 - la clé aléatoire est "WMCKL"
 
 On additionne chaque lettre du mot avec chaque lettre de la clé (module 26)

```
   7 (H)   4 (E)  11 (L)  11 (L)  14 (O) message
+ 22 (W)  12 (M)   2 (C)  10 (K)  11 (L) masque
= 29      16      13      21      25     masque + message
=  3 (D)  16 (Q)  13 (N)  21 (V)  25 (Z) masque + message  modulo 26
``` 

Le message chiffré est donc **DQNVZ**

Pour déchiffrer on soustrait le masque au texte chiffré.

```
    3 (D)  16 (Q)  13 (N)  21 (V)  25 (Z) message chiffré
-  22 (W)  12 (M)   2 (C)  10 (K)  11 (L) masque
= -19       4      11      11      14     message chiffré - masque
=   7 (H)   4 (E)  11 (L)  11 (L)  14 (O) message chiffré - masque modulo 26
``` 

#### Chiffrement par l'opérateur XOR

Dans les messages numériques on applique un décalage modulo 2 ce qui équivaut à utiliser l'opérateur **XOR**.

Le principe est le suivant: On possède un texte T à encoder et un texte clé C.
Si la clé est plus courte que le message on répète la clé.

Pour chaque code (ascii, utf-8...) du texte on effectue l'operation XOR avec le code de la lettre de la clé.

**Rappel:**
La table logique XOR (ou exclusif) est la suivante.

<table style="border:1px solid black">
    <thead>
        <tr>
            <th colspan="3">Table de vérité de l'opérateur OU EXCLUSIF</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="border:1px solid black">a</td>
            <td style="border:1px solid black">b</td>
            <td style="border:1px solid black">out=a⊕b</td>
        </tr>
        <tr>
            <td style="border:1px solid black">0</td>
            <td style="border:1px solid black">0</td>
            <td style="border:1px solid black">0</td>
        </tr>
        <tr>
            <td style="border:1px solid black">0</td>
            <td style="border:1px solid black">1</td>
            <td style="border:1px solid black">1</td>
        </tr>
        <tr>
            <td style="border:1px solid black">1</td>
            <td style="border:1px solid black">0</td>
            <td style="border:1px solid black">1</td>
        </tr>
        <tr>
            <td style="border:1px solid black">1</td>
            <td style="border:1px solid black">1</td>
            <td style="border:1px solid black">0</td>
        </tr>
    </tbody>
</table>


**Exemple**

Le texte à coder est "Bonjour".
La clé est "NSI"

Effectuons le code sur la première lettre du message et la première lettre de la clé.

In [50]:
mot = "Bonjour"
cle = "nstgjeu"

lettre1_mot = mot[0]
lettre1_cle = cle[0]

print("Encodage de la première lettre.")
print("-------------------------------")
print(f"{lettre1_mot} -> {ord(lettre1_mot)} ->  {bin(ord(lettre1_mot))}")
print(f"{lettre1_cle} -> {ord(lettre1_cle)} -> {bin(ord(lettre1_cle))}")

print("Effectuons un XOR")

result = ord(lettre1_mot) ^ ord(lettre1_cle)
print(f"{lettre1_mot} ^ {lettre1_cle} => {result} soit {chr(result)}")

print("Décodage")
print("--------")

decode = ord(chr(result)) ^ ord(lettre1_cle)

print(f"{chr(result)} ^ {lettre1_cle} soit {result} ^ {ord(lettre1_cle)} -> {decode} c'est à dire la lettre {chr(decode)}")

Encodage de la première lettre.
-------------------------------
B -> 66 ->  0b1000010
n -> 110 -> 0b1101110
Effectuons un XOR
B ^ n => 44 soit ,
Décodage
--------
, ^ n soit 44 ^ 110 -> 66 c'est à dire la lettre B


Le décodage est possible grâce à la **réversibilité** de l'opérateur XOR.

<table style="border:1px solid black">
    <thead>
        <tr>
            <th colspan="3">Réversibilité de l'opérateur XOR </th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="border:1px solid black">T</td>
            <td style="border:1px solid black">C</td>
            <td style="border:1px solid black">T chiffré = T⊕C</td>
            <td style="border:1px solid black">T déchiffré = (T⊕C)⊕C = T</td>
        </tr>
        <tr>
            <td style="border:1px solid black"><b>0</b></td>
            <td style="border:1px solid black">0</td>
            <td style="border:1px solid black">0</td>
            <td style="border:1px solid black"><b>0</b></td>
        </tr>
        <tr>
            <td style="border:1px solid black"><b>0</b></td>
            <td style="border:1px solid black">1</td>
            <td style="border:1px solid black">1</td>
            <td style="border:1px solid black"><b>0</b></td>
        </tr>
        <tr>
            <td style="border:1px solid black"><b>1</b></td>
            <td style="border:1px solid black">0</td>
            <td style="border:1px solid black">1</td>
            <td style="border:1px solid black"><b>1</b></td>
        </tr>
        <tr>
            <td style="border:1px solid black"><b>1</b></td>
            <td style="border:1px solid black">1</td>
            <td style="border:1px solid black">0</td>
            <td style="border:1px solid black"><b>1</b></td>
        </tr>
    </tbody>
</table>

**Exercice1** : Démontrer algébriquement que (a⊕b)⊕b = a.

**Exercice2** : 

- Ecrire une *fonction encode_xor(text, cle)* qui prend en paramètre un texte (string) et une clé (texte) et qui retourne une chaine de caractère codée par le chiffrement l'opérateur XOR.
- Ecrire une *fonction decode_xor(text, cle)* qui prend en paramètre un texte codé (string) et une clé (string) et qui retourne la chaine de caractère décodée.

Afin de limiter la difficulté, nous utiliserons uniquement des caractères ASCII dans le texte et la clé.

**Remarque** : Dans la pratique, la clé n'est pas aussi longue que le message, elle est **étendue** par répétition.



#### Algorithme AES

AES (*Advanced Encryption Standard*) est un des algorithmes de chiffrement symétrique les plus utilisés.

[Voir l'article Wikipédia](https://fr.wikipedia.org/wiki/Advanced_Encryption_Standard)

Il ressemble au chiffrement XOR tout en étant plus complexe.

- la clé initiale est étendue mais pas par simple répétition
- le message et clé sont mélangés par un opérateur réversible différent de ⊕.

## Chiffrement asymétrique

### Principe du protocole d'échange de clés de Diffie-Hellman

Le principe du chiffrement asymétrique a été créé par les mathématiciens Whitfield Diffie et Martin Hellman en 1976. Ils ont reçu le prix Turing en 2015 pour cette invention.

<figure>
<img src="img/hellman_diffie.jpg" alt="Martin Hellman et Whitfield Diffie" title="Martin Hellman et Whitfield Diffie" width="33%" style="border:1px solid black;">
<figcaption>Martin Hellman et Whitfield Diffie</figcaption>
</figure>

<img src="img/peinture.png" alt="Analogie des pots de peinture pour l'echange de clés" title="Analogie des pots de peinture pour l'echange de clés" width="30%" style="float:right">

L'analogie la plus pour expliquer est celle des pots de peinture.

- Alice et Bob choisissent une couleur commune et la partage publiquement;
- Chacun choisit une couleur secrète et la mélange avec la couleur publique;
- Ils s'échangent publiquement les couleurs obtenues;
- Chacun mélange sa couleur secrète avec le mélange de son interlocuteur, ils obtiennent donc chacun une clé commune.

Mathématiquement, ce mélange est une fonction M à deux variables telle que:

- Si on connait $M(x,y)$ et $x$ alors on ne peut pas retrouver $y$ ( ou alors très difficilement) 
- Pour tous nombres x, y et z $M(M(x,y), z) = M(M(z, x), y)$ c'est la clé commune.
  - x est ici la couleur publique
  - y la couleur secrète d'Alice
  - z la couleur secrète de Bob
  - M(x, y) est la couleur mélangée d'Alice échangée publiquement à Bob
  - M(x, z) est la couleur mélangée de Bob échangée publiquement à Alice
  - M(M(x,y), z) et M(M(z, x), y) sont leurs deux clés qui sont égales.

Un inconvénient subsiste : cette méthode ne contient **aucune authentification** des participants.

### Le chiffrement RSA

<figure style="float:right">
<img src="img/rsa_inventors.jpg" alt="Ronald Rivest, Adi Shamir, and Leonard Adleman" title="Ronald Rivest, Adi Shamir, and Leonard Adleman" style="border:1px solid black;">
<figcaption>Ronald Rivest, Adi Shamir et Leonard Adleman les inventeurs du chiffrement RSA.</figcaption>
</figure>

Aujourd'hui, l'algorithme de chiffrement le plus utilisé est le [chiffrement RSA](https://fr.wikipedia.org/wiki/Chiffrement_RSA). Inventé en 1978, l'algorithme de chiffrement est dans le domaine public depuis 2000 et peut donc être utilisé gratuitement.

Mathématiquement, l'algorithme se base sur les **nombres premiers**, la **congruence** et le **petit théorème de Fermat**.

Voir [un exemple sur Wikipedia](https://fr.wikipedia.org/wiki/Chiffrement_RSA#Exemple).


## HTTPS

Nous avons vu que le principe de Diffie-Hellman ne propose pas de système d'authentification. Si une personne malveillante (Eve) intercepte les communications d'Alice et Bob elle peut se faire passer pour l'autre pour chacun de ces deux protaganistes. C'est ce qu'on appelle l'**attaque de l'homme du milieu**.

Afin d'authentifier les serveurs lors des connections https, on demande à ces serveurs de fournir un **certificat numérique**. Ces certificat sont fournis aux administrateur d'un site par des **autorités de certification** (*Certificate Authority :CA*). L'autorité de certification fournit des moyens pour vérifier la validité de ces certificats.

*https* utilise le protocole **TLS** (Transport Layer Security) qui utilise deux types de chiffrement:
 - Un chiffrement symétrique (AES) à l'aide d'une clé secrète
 - le chiffrement asymétrique (RSA) pour l’échange en toute sécurité de cette clé secrète
 
 <figure style="float:right">
<img src="img/TLS_certification_process3.png" alt="Principe d'une communication en https" title="Principe d'une communication en https" style="border:1px solid black;" width="50%">
<figcaption>Principe d'une communication en https</figcaption>
</figure>


In [1]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/1Yv8m398Fv0?si=JV-n3QLZqLSOunAz" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

In [2]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/7W7WPMX7arI?si=jlYWs-0bHQUdPTeY" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

[Article Wikipedia sur le chiffrement asymétrique](https://fr.wikipedia.org/wiki/Cryptographie_asym%C3%A9trique)

[Accueil](../../../index.ipynb) > [Sommaire de Terminale](../../index.ipynb)